In [1]:
import logging
from pathlib import Path

from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

data_dir = Path("/projects/dsgrid/tempo-project/april-queries/")
dataset_name = "state_level_simplified"

spark = (
            SparkSession.builder
            .appName("dsgrid")
            .config("spark.sql.session.timeZone", "EST")
            .getOrCreate()
        )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/30 15:15:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.parquet(str(data_dir / dataset_name / "table.parquet"))
df.createOrReplaceTempView("tbl")
df.show(n=5)

+-----+-------------------------+-----------+-------------------+------------+------------------+------------+
|state|tempo_project_model_years|  subsector|           time_est|weather_2012|             value|    scenario|
+-----+-------------------------+-----------+-------------------+------------+------------------+------------+
|   CA|                     2036|phev_pickup|2012-09-28 16:00:00|        2012| 275.1090731349142|efs_high_ldv|
|   CA|                     2036|phev_pickup|2012-04-26 16:00:00|        2012|303.16233773326866|efs_high_ldv|
|   CA|                     2036|phev_pickup|2012-05-31 14:00:00|        2012|200.52595789310362|efs_high_ldv|
|   CA|                     2036|phev_pickup|2012-06-08 22:00:00|        2012|194.36912422169758|efs_high_ldv|
|   CA|                     2036|phev_pickup|2012-11-07 13:00:00|        2012|213.74852081008243|efs_high_ldv|
+-----+-------------------------+-----------+-------------------+------------+------------------+------------+
o

In [7]:
df2 = spark.sql(f"SELECT * FROM tbl WHERE (tempo_project_model_years = 2050) AND (state = 'CO') ORDER BY subsector, time_est LIMIT 5;")
df2.show()

+-----+-------------------------+-----------+-------------------+------------+------------------+------------------+
|state|tempo_project_model_years|  subsector|           time_est|weather_2012|             value|          scenario|
+-----+-------------------------+-----------+-------------------+------------+------------------+------------------+
|   CO|                     2050|bev_compact|2012-01-01 00:00:00|        2012|184.78713143567745|ldv_sales_evs_2035|
|   CO|                     2050|bev_compact|2012-01-01 00:00:00|        2012|47.780491590023026|         reference|
|   CO|                     2050|bev_compact|2012-01-01 00:00:00|        2012|161.75238932508225|      efs_high_ldv|
|   CO|                     2050|bev_compact|2012-01-01 01:00:00|        2012|34.985022796630865|         reference|
|   CO|                     2050|bev_compact|2012-01-01 01:00:00|        2012|139.11690390121947|ldv_sales_evs_2035|
+-----+-------------------------+-----------+-------------------